In [5]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="wAYldlHqvi659XERmPPL")
project = rf.workspace("zaid-ahmed-aaaas").project("classification-of-name-plates-24xpg")
version = project.version(1)
dataset = version.download("yolov8")

dataset_path = dataset.location  # <- this is the correct path to use
print("Dataset location:", dataset_path)

loading Roboflow workspace...
loading Roboflow project...
Dataset location: c:\Users\Aosaf\Python Files\Q1 Broken Plate detection\classification-of-name-plates-1


In [6]:
from ultralytics import YOLO

# Initialize YOLOv8 model (nano version for fast experiments)
model = YOLO('yolov8n.pt')  

# Train on your dataset
model.train(
    data=f"{dataset_path}/data.yaml",  # Roboflow provides this YAML
    epochs=100,                         # start with 50 epochs
    imgsz=640,                         # image size
    batch=16,                           # adjust to your GPU
    name="license_plate_detector"
)


Ultralytics 8.3.203  Python-3.13.7 torch-2.8.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=c:\Users\Aosaf\Python Files\Q1 Broken Plate detection\classification-of-name-plates-1/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=license_plate_detector, nbs=64, nms=False, opse

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001A68AA5F310>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0

In [13]:
import os
# Use raw string for Windows paths
weights_path = r"C:\Users\Aosaf\Python Files\Q1 Broken Plate detection\runs\train\license_plate_detector\weights\best.pt"

# Check if file exists
if not os.path.exists(weights_path):
    print("Weights file not found! Make sure you trained the model first.")
else:
    model = YOLO(weights_path)



Weights file not found! Make sure you trained the model first.


In [15]:
fr_image_path = r"pic1.jpg"
re_image_path = r"pic1.jpg"

# Check files
assert os.path.exists(fr_image_path), "FR image not found!"
assert os.path.exists(re_image_path), "RE image not found!"


In [16]:
def preprocess_plate(image, box):
    x1, y1, x2, y2 = map(int, box)
    plate = image[y1:y2, x1:x2]
    gray = cv2.cvtColor(plate, cv2.COLOR_BGR2GRAY)
    gray = cv2.bilateralFilter(gray, 9, 75, 75)
    gray = cv2.equalizeHist(gray)
    th = cv2.adaptiveThreshold(gray, 255,
                               cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                               cv2.THRESH_BINARY_INV, 15, 10)
    return th

def segment_characters(bin_plate):
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    clean = cv2.morphologyEx(bin_plate, cv2.MORPH_OPEN, kernel)
    contours, _ = cv2.findContours(clean, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    boxes = []
    for c in contours:
        x,y,w,h = cv2.boundingRect(c)
        if 10 < h < bin_plate.shape[0] and 5 < w:
            boxes.append((x,y,w,h))
    boxes = sorted(boxes, key=lambda b: b[0])
    return boxes


In [17]:
def is_broken_char(bin_char):
    h, w = bin_char.shape
    fg_ratio = (bin_char>0).sum() / (w*h)
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(bin_char, connectivity=8)
    areas = stats[1:, cv2.CC_STAT_AREA] if num_labels>1 else []
    largest = areas.max() if len(areas) > 0 else 0
    largest_ratio = largest / (w*h)
    if fg_ratio < 0.10 or largest_ratio < 0.06 or num_labels-1 > 3:
        return True, dict(fg_ratio=fg_ratio, num_cc=num_labels-1, largest_ratio=largest_ratio)
    return False, dict(fg_ratio=fg_ratio, num_cc=num_labels-1, largest_ratio=largest_ratio)


In [18]:
def process_vehicle(image_path, model):
    img = cv2.imread(image_path)
    results = model.predict(image_path)[0]
    plates = results.boxes.xyxy.cpu().numpy()
    vehicle_results = []
    for box in plates:
        bin_plate = preprocess_plate(img, box)
        char_boxes = segment_characters(bin_plate)
        chars = []
        for (cx,cy,cw,ch) in char_boxes:
            cimg = bin_plate[cy:cy+ch, cx:cx+cw]
            broken, meta = is_broken_char(cimg)
            chars.append({'box':(cx,cy,cw,ch), 'broken':broken, 'meta':meta})
        vehicle_results.append({'plate_box':box, 'chars':chars})
    vehicle_flagged = any(c['broken'] for r in vehicle_results for c in r['chars'])
    return vehicle_results, vehicle_flagged

# Run on FR + RE
fr_results, fr_flag = process_vehicle(fr_image_path, model)
re_results, re_flag = process_vehicle(re_image_path, model)
vehicle_flagged = fr_flag or re_flag
print("Vehicle has broken characters?", vehicle_flagged)



image 1/1 c:\Users\Aosaf\Python Files\Q1 Broken Plate detection\pic1.jpg: 512x640 1 front, 32.9ms
Speed: 4.1ms preprocess, 32.9ms inference, 12.9ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 c:\Users\Aosaf\Python Files\Q1 Broken Plate detection\pic1.jpg: 512x640 1 front, 4.3ms
Speed: 1.7ms preprocess, 4.3ms inference, 1.3ms postprocess per image at shape (1, 3, 512, 640)
Vehicle has broken characters? False
